<a href="https://colab.research.google.com/github/yasin-sazid/university-schedule-optimizer/blob/main/university_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from ortools.sat.python import cp_model
import pandas as pd

# Load Excel files
classrooms_df = pd.read_excel("data.xlsx", sheet_name="classrooms")
slots_df = pd.read_excel("data.xlsx", sheet_name="slots")
schedule_df = pd.read_excel("sample_schedule_100.xlsx", sheet_name="Sheet1")

# Step 1: Clean and tag schedule data
valid_schedule = schedule_df[schedule_df['Course'].notna() & schedule_df['Instructor'].notna()].copy()
valid_schedule['CourseID'] = valid_schedule['Course'].astype(str) + "_" + valid_schedule['Section'].astype(str)
valid_schedule['IsLab'] = valid_schedule['Course'].str.contains(r'Lab', case=False, na=False)

valid_schedule['Time From'] = pd.to_datetime(valid_schedule['Time From']).dt.strftime("%H:%M:%S")
valid_schedule['Time To']   = pd.to_datetime(valid_schedule['Time To']).dt.strftime("%H:%M:%S")

# Step 1.5: Expand Weekday column into list of days
def expand_days(day_str):
    return list(day_str.strip()) if isinstance(day_str, str) else []

valid_schedule['Days'] = valid_schedule['Weekday'].apply(expand_days)
valid_schedule['NumDays'] = valid_schedule['Days'].apply(len)

# Room and time slot encoding
room_list = classrooms_df['Room'].dropna().unique().tolist()
room_index = {room: idx for idx, room in enumerate(room_list)}
room_type_map = dict(zip(classrooms_df['Room'], classrooms_df['Theory/Lab']))
room_size_map = dict(zip(classrooms_df['Room'], classrooms_df['Size']))

slots_df['TimeSlot'] = (
    slots_df['Time From'].astype(str) + "-" +
    slots_df['Time To'].astype(str) + "-" +
    slots_df['Weekday']
)

print(slots_df.groupby(['Weekday', 'SlotType']).size())

timeslot_list = slots_df['TimeSlot'].unique().tolist()
timeslot_index = {slot: idx for idx, slot in enumerate(timeslot_list)}
slot_types = slots_df['SlotType'].tolist()

print("\n🧾 Course Summary:")
for _, row in valid_schedule.iterrows():
    print(f"• {row['CourseID']}: Days={row['Days']}, IsLab={row['IsLab']}, Size={row['Size']}, Instructor={row['Instructor']}")

# Step 2: Create variables
model = cp_model.CpModel()
room_vars = {}
timeslot_vars = {}

for _, row in valid_schedule.iterrows():
    cid = row['CourseID']
    days = row['Days']

    room_vars[cid] = model.NewIntVar(0, len(room_list) - 1, f"room_{cid}")
    timeslot_vars[cid] = [
        model.NewIntVar(0, len(timeslot_list) - 1, f"time_{cid}_{d}")
        for d in days
    ]

# Step 3: Time slot filtering based on type and day
from datetime import datetime

# Map timeslot index to its day and slot type
slot_info = {
    idx: {
        'day': ts.split('-')[-1],
        'slot_type': slots_df.iloc[idx]['SlotType']
    }
    for idx, ts in enumerate(timeslot_list)
}

# Step 3: Timeslot must match correct day and slot type

from datetime import datetime, timedelta

def parse_time_duration(t_from, t_to):
    try:
        t1 = pd.to_datetime(t_from).time()
        t2 = pd.to_datetime(t_to).time()

        start = datetime.combine(datetime.today(), t1)
        end = datetime.combine(datetime.today(), t2)

        # If end is earlier, it's probably PM (Excel time logic)
        if end <= start:
            end = datetime.combine(datetime.today(), t2) + timedelta(hours=12)

        duration = (end - start).seconds / 3600
        return duration
    except Exception as e:
        print(f"❌ Failed to parse duration from {t_from} to {t_to}: {e}")
        return 0

from datetime import datetime

slot_info = {
    idx: {
        'day': ts.split('-')[-1],
        'slot_type': slots_df.iloc[idx]['SlotType']
    }
    for idx, ts in enumerate(timeslot_list)
}

for _, row in valid_schedule.iterrows():
    cid = row['CourseID']
    days = row['Days']
    is_lab = row['IsLab']
    time_from = str(row['Time From']).strip()
    time_to = str(row['Time To']).strip()

    try:
        duration_hours = parse_time_duration(time_from, time_to)
    except:
        duration_hours = 0

    for i, d in enumerate(days):
        allowed_types = []

        if is_lab:
            if 2.8 <= duration_hours <= 3.2:
                allowed_types = ['Lab-3hr']
            elif 1.7 <= duration_hours <= 2.3:
                allowed_types = ['Lab-2hr']
            elif 1.3 <= duration_hours <= 1.7:
                allowed_types = ['Theory']
        else:
            allowed_types = ['Theory']

        valid_indices = [
            idx for idx, info in slot_info.items()
            if info['day'] == d and info['slot_type'] in allowed_types
        ]

        if not valid_indices:
            print(f"⚠️ Fallback: No matching slot for {cid} on {d} (duration={duration_hours:.2f}h), relaxing...")
            valid_indices = [
                idx for idx, info in slot_info.items()
                if info['day'] == d
            ]

        model.AddAllowedAssignments([timeslot_vars[cid][i]], [[j] for j in valid_indices])


# Step 4: Room-time clash prevention
course_ids = list(valid_schedule['CourseID'])

for i in range(len(course_ids)):
    for j in range(i + 1, len(course_ids)):
        cid1, cid2 = course_ids[i], course_ids[j]

        for idx_a, t1 in enumerate(timeslot_vars[cid1]):
            for idx_b, t2 in enumerate(timeslot_vars[cid2]):
                day_a = valid_schedule.loc[valid_schedule['CourseID'] == cid1, 'Days'].values[0][idx_a]
                day_b = valid_schedule.loc[valid_schedule['CourseID'] == cid2, 'Days'].values[0][idx_b]

                if day_a == day_b:
                    same_room = model.NewBoolVar(f"{cid1}_{cid2}_same_room_{day_a}")
                    same_time = model.NewBoolVar(f"{cid1}_{cid2}_same_time_{day_a}")

                    model.Add(room_vars[cid1] == room_vars[cid2]).OnlyEnforceIf(same_room)
                    model.Add(room_vars[cid1] != room_vars[cid2]).OnlyEnforceIf(same_room.Not())

                    model.Add(t1 == t2).OnlyEnforceIf(same_time)
                    model.Add(t1 != t2).OnlyEnforceIf(same_time.Not())

                    model.AddBoolOr([same_room.Not(), same_time.Not()])

# Step 5: Instructor conflict
instructor_courses = valid_schedule.groupby('Instructor')['CourseID'].apply(list).to_dict()
print("\n👨‍🏫 Instructor Course Mapping:")
for instructor, cids in instructor_courses.items():
    print(f"• {instructor} teaches {len(cids)} courses → {cids}")
    if len(cids) > 5:
        print(f"⚠️ Instructor {instructor} has high teaching load")
    for i in range(len(cids)):
        for j in range(i + 1, len(cids)):
            cid1, cid2 = cids[i], cids[j]
            for t1 in timeslot_vars[cid1]:
                for t2 in timeslot_vars[cid2]:
                    model.Add(t1 != t2)

# Step 6: Room capacity
print("\n🏫 Valid Room Mapping for Capacity:")
for _, row in valid_schedule.iterrows():
    cid = row['CourseID']
    size = int(row['Size'])
    valid_room_indices = [
        room_index[room_name]
        for room_name, capacity in room_size_map.items()
        if pd.notna(capacity) and int(capacity) >= size
    ]
    print(f"• {cid} needs size {size} → {len(valid_room_indices)} valid rooms")
    if not valid_room_indices:
        print(f"❌ No rooms with sufficient capacity for {cid}")
    model.AddAllowedAssignments([room_vars[cid]], [[r] for r in valid_room_indices])

# Step 7: Lab room constraint
print("\n🔬 Lab Room Mapping:")
for _, row in valid_schedule.iterrows():
    cid = row['CourseID']
    if row['IsLab']:
        valid_lab_indices = [
            room_index[r] for r in room_type_map
            if isinstance(room_type_map[r], str) and 'lab' in room_type_map[r].lower()
        ]
        print(f"• {cid} is a lab → {len(valid_lab_indices)} candidate rooms")
        if not valid_lab_indices:
            print(f"❌ No lab rooms found for {cid}")
        model.AddAllowedAssignments([room_vars[cid]], [[r] for r in valid_lab_indices])

        from datetime import datetime

def get_duration_hours(row):
    try:
        fmt = "%H:%M"
        start_dt = datetime.strptime(str(row['Time From']).strip(), fmt)
        end_dt = datetime.strptime(str(row['Time To']).strip(), fmt)
        return (end_dt - start_dt).seconds / 3600
    except:
        return 0

valid_schedule['DurationHrs'] = valid_schedule.apply(get_duration_hours, axis=1)

# For lab courses
lab_durations = valid_schedule[valid_schedule['IsLab']]

print("Lab duration breakdown:")
print(lab_durations['DurationHrs'].value_counts(bins=[0, 1.7, 2.3, 3.2], sort=False))

# Track all penalties to minimize
penalty_vars = []
penalty_weights = []  # Optional: can assign different weights to room vs time

for _, row in valid_schedule.iterrows():
    cid = row['CourseID']
    preferred_room = row['Room'] if pd.notna(row['Room']) else None
    preferred_time_from = row['Time From'] if pd.notna(row['Time From']) else None
    preferred_time_to = row['Time To'] if pd.notna(row['Time To']) else None
    days = row['Days']

    # 🏫 ROOM PREFERENCE PENALTY
    if preferred_room in room_index:
        pref_room_idx = room_index[preferred_room]
        room_match = model.NewBoolVar(f"room_pref_match_{cid}")
        model.Add(room_vars[cid] == pref_room_idx).OnlyEnforceIf(room_match)
        model.Add(room_vars[cid] != pref_room_idx).OnlyEnforceIf(room_match.Not())
        penalty_vars.append(room_match.Not())  # 1 if not matched
        penalty_weights.append(50)  # e.g., room preference penalty weight

    # 🕒 TIME PREFERENCE PENALTY (match any day slot)
    try:
        from_str = pd.to_datetime(preferred_time_from).strftime("%H:%M:%S")
        to_str = pd.to_datetime(preferred_time_to).strftime("%H:%M:%S")
        time_str = f"{from_str}-{to_str}"

        matched_indices = [
            idx for idx, slot in enumerate(timeslot_list)
            if slot.startswith(time_str) and slot.split("-")[-1] in days
        ]

        if not matched_indices:
            print(f"❌ No matching slot found for {cid}: preferred {time_str}, days {days}")
        else:
            print(f"✅ {cid} → Preferred {time_str} matched to slots {[timeslot_list[i] for i in matched_indices]}")


        if matched_indices:
            for i, t_var in enumerate(timeslot_vars[cid]):
                time_match = model.NewBoolVar(f"time_pref_match_{cid}_{i}")
                model.AddAllowedAssignments([t_var], [[idx] for idx in matched_indices]).OnlyEnforceIf(time_match)
                model.AddForbiddenAssignments([t_var], [[idx] for idx in matched_indices]).OnlyEnforceIf(time_match.Not())
                penalty_vars.append(time_match.Not())  # 1 if not matched
                penalty_weights.append(30)  # e.g., time preference penalty weight
    except:
        pass

# 🧠 MINIMIZE TOTAL PENALTIES
model.Minimize(sum(w * v for w, v in zip(penalty_weights, penalty_vars)))


# Step 8: Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"\n📊 Solver status: {solver.StatusName(status)}")
print(f"🕒 Wall time: {solver.WallTime():.2f}s")
print(f"🔁 Conflicts: {solver.NumConflicts()}")
print(f"↪️ Branches: {solver.NumBranches()}")
print("🧠 Total courses scheduled:", len(result_df['Course'].unique()))
print("📚 Total slots used:", len(result_df))
print("🏫 Room usage breakdown:\n", result_df['Room'].value_counts())
# Only one solution attempted in basic solve, SolutionCount is not available.
print(f"📏 (Note: Only one solution attempted — use a solution callback to enumerate more)")

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    records = []
    for _, row in valid_schedule.iterrows():
        cid = row['CourseID']
        section = row['Section']
        course = row['Course']
        actual_room = room_list[solver.Value(room_vars[cid])]
        preferred_room = row['Room'] if pd.notna(row['Room']) else None

        for i, d in enumerate(row['Days']):
            ts_index = solver.Value(timeslot_vars[cid][i])
            ts_string = timeslot_list[ts_index]
            time_from, time_to, day = ts_string.split('-')

            pref_time_from = pd.to_datetime(row['Time From']).strftime("%H:%M:%S")
            pref_time_to   = pd.to_datetime(row['Time To']).strftime("%H:%M:%S")

            pref_time_match = (
                pref_time_from == time_from and
                pref_time_to == time_to
            )

            pref_room_match = (
                preferred_room == actual_room
            )


            if not pref_time_match:
                print(f"❌ {cid} did NOT get preferred slot → Time: {row['Time From']}–{row['Time To']} ≠ {time_from}–{time_to}")
            else:
                print(f"✅ {cid} got preferred slot → Time: {row['Time From']}–{row['Time To']} = {time_from}–{time_to}")
            if not pref_room_match:
                print(f"❌ {cid} did NOT get preferred slot → Room: {preferred_room} ≠ {actual_room}")
            else:
                print(f"✅ {cid} got preferred slot → Room: {preferred_room} = {actual_room}")

            # 📝 Final schedule entry
            records.append({
                'Course': course,
                'Section': section,
                'Instructor': row['Instructor'],
                'Day': d,
                'Room': actual_room,
                'Time From': time_from,
                'Time To': time_to
            })

    result_df = pd.DataFrame(records)
    result_df.to_excel("final_schedule_output.xlsx", index=False)
    print("✅ Output written to final_schedule_output.xlsx")
else:
    print("❌ No feasible solution found.")
    # with open("model_dump.txt", "w") as f:
    #     f.write(str(model))
    # print("📄 Model structure dumped to model_dump.txt")

/tmp/ipython-input-24-3763053651.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  valid_schedule['Time From'] = pd.to_datetime(valid_schedule['Time From']).dt.strftime("%H:%M:%S")
/tmp/ipython-input-24-3763053651.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  valid_schedule['Time To']   = pd.to_datetime(valid_schedule['Time To']).dt.strftime("%H:%M:%S")


Weekday  SlotType
M        Lab-2hr     4
         Lab-3hr     4
         Theory      7
R        Lab-2hr     4
         Lab-3hr     4
         Theory      7
S        Lab-2hr     4
         Lab-3hr     4
         Theory      7
T        Lab-2hr     4
         Lab-3hr     4
         Theory      7
W        Lab-2hr     4
         Lab-3hr     4
         Theory      7
dtype: int64

🧾 Course Summary:
• ACT101_1: Days=['M', 'W'], IsLab=False, Size=30, Instructor=MRDI
• ACT101_2: Days=['T', 'R'], IsLab=False, Size=30, Instructor=NHN
• ACT101_3: Days=['S', 'T'], IsLab=False, Size=30, Instructor=NHN
• ACT101_4: Days=['S', 'R'], IsLab=False, Size=30, Instructor=THD
• ACT101_5: Days=['S', 'T'], IsLab=False, Size=30, Instructor=THD
• ACT101_6: Days=['M', 'W'], IsLab=False, Size=30, Instructor=THD
• ACT101_7: Days=['M', 'W'], IsLab=False, Size=30, Instructor=MSQI
• ACT101_8: Days=['S', 'T'], IsLab=False, Size=30, Instructor=MSQI
• ACT101_9: Days=['M', 'W'], IsLab=False, Size=30, Instructor=MHRN
• ACT10